<div style="text-align: center;">
    <span style="color: #000000; font-size: 30px; font-weight: bold; font-family: Arial;  margin: 50px; ">
        Final Assignment Part Two
    </span>
</div>

#### <centre>form an optimal portfolio and reblance it</centre>


Choose a collection of assets. Using simulation, find the weights that will lead to the minimum variance portfolio. Make a python function that will purchase the correct amount of assets on Alpaca.

In [4]:
pip install alpaca-trade-api

Note: you may need to restart the kernel to use updated packages.


## import packages


In [5]:
import yfinance as yf
import numpy as np
from scipy.optimize import minimize
import requests
import json
import alpaca_trade_api as tradeapi

# Choose a collection of assets

In [6]:
tickers = ['SLB',  'OXY', 'BP',  'WMB',  'BKR', 'APA', 'HES']

# Step 1 Get Optimal Weights


In [7]:
# 获取股票数据
# get stock data
data = yf.download(tickers, start='2000-01-01', end='2023-09-07')['Adj Close']

# 计算每只股票的日收益率
# daily return of each stock
returns = data.pct_change().dropna()

# 计算协方差矩阵
#covarince calculation
cov_matrix = returns.cov()

# 定义目标函数，最小化方差
# target function, minimize variance
def objective(weights):
    portfolio_variance = np.dot(weights.T, np.dot(cov_matrix, weights))
    return portfolio_variance

# 设置约束条件，这里假设权重之和为1
# set the constraints
constraints = ({'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1})

# 设置初始权重猜测值
#set paramters' initial values
#first I try
#   initial_weights = np.ones(len(tickers)) / len(tickers)
#then I find random renders better effect

initial_weights = np.random.rand(len(tickers))
initial_weights /= initial_weights.sum() 

# 最小化目标函数，使用不同的优化方法
# minimize
result = minimize(objective, initial_weights, constraints=constraints, method='trust-constr')


# 最优权重
optimal_weights = result.x

print("Optimal weights：", optimal_weights)
print("minimum variance：", result.fun)


[*********************100%%**********************]  7 of 7 completed


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/scipy/optimize/_hessian_update_strategy.py:182: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  warn('delta_grad == 0.0. Check if the approximated '


Optimal weights： [0.17825812 0.08328665 0.1555301  0.09430659 0.18928132 0.12381588
 0.17552133]
minimum variance： 0.00047309071867106923


# Step 2 Get Account Balance

In [8]:
#与羊驼关联
#connect with alpaca market website
API_KEY = "PKOK1AECOYSVDXHMQ7ON"
SECRET_KEY = "xeUuUHj4TuLJ95NaSSMbdKRWERyPGJofYbdoioki"
BASE_URL = "https://paper-api.alpaca.markets"
ACCOUNT_URL = BASE_URL + "/v2/account"
ORDERS_URL =  BASE_URL + "/v2/orders"

In [9]:
# request based on keys and endpoints above
r = requests.get(ACCOUNT_URL, headers = {'APCA-API-KEY-ID': API_KEY,
                                         'APCA-API-SECRET-KEY':  SECRET_KEY})
info = json.loads(r.content)
accountval = float(info["cash"])

# Step 3 Get number of Shares

In [10]:
# 计算每个股票的股票数量
# calculate the portion of each stocks in the portfolio
shares = []
for i, symbol in enumerate(tickers):
    weight = optimal_weights[i]
    price = data[symbol][-1]  # 获取最后一个交易日的股价
    qty = (weight * accountval) / price
    qty = int(qty)  # 取整数部分
    shares.append(qty)

for i, symbol in enumerate(tickers):
    print(f"The optimal number of stock {symbol} : {shares[i]}")

The optimal number of stock SLB : -1620
The optimal number of stock OXY : -706
The optimal number of stock BP : -2220
The optimal number of stock WMB : -1565
The optimal number of stock BKR : -2819
The optimal number of stock APA : -1537
The optimal number of stock HES : -618


# Step 4 Purchase amounts

In [11]:
def create_order(symbol, qty, side, type_, time_in_force):
    data = {
            "symbol": symbol,
            "qty": qty,
            "side": side,
            "type": type_,
            "time_in_force": time_in_force
            }
    r = requests.post(ORDERS_URL, json=data, headers = {'APCA-API-KEY-ID': API_KEY, 
                                                        'APCA-API-SECRET-KEY':  SECRET_KEY})

    return json.loads(r.content)


### REBALANCE PORTFOLIO TO CHECK THAT THE WEIGHTS ARE ALIGNED EACH DAY

In [12]:
# 计算当前股票数量
# present stock price
current_shares = []
for i, symbol in enumerate(tickers):
    latest_prices = data.iloc[-1]
    price = latest_prices[symbol]
    qty = (accountval / len(tickers)) / price
    qty = int(qty)  # 取整数部分
    current_shares.append(qty)

# 打印当前股票数量
for i, symbol in enumerate(tickers):
    print(f"The current number of stock {symbol} : {current_shares[i]}")

# 计算最优股票数量与当前股票数量之间的差异

#  calculate the difference of present quantity and optimal quantity:
#     present quantity > optimal quantity : sell
#     present quantity < optimal quantity : buy

difference = [shares[i] - current_shares[i] for i in range(len(tickers))]

# 打印差异

for i, symbol in enumerate(tickers):
    print(f"The difference of stock {symbol} : {difference[i]}")

The current number of stock SLB : -1298
The current number of stock OXY : -1211
The current number of stock BP : -2039
The current number of stock WMB : -2371
The current number of stock BKR : -2128
The current number of stock APA : -1773
The current number of stock HES : -503
The difference of stock SLB : -322
The difference of stock OXY : 505
The difference of stock BP : -181
The difference of stock WMB : 806
The difference of stock BKR : -691
The difference of stock APA : 236
The difference of stock HES : -115


In [13]:
# Execute Trades to accoujnt for the difference
for i, symbol in enumerate(tickers):
    qty_diff = difference[i]
    if qty_diff < 0:
        # 卖出股票
        # present quantity > optimal quantity : sell
        
        create_order(symbol, abs(qty_diff), "sell", "market", "day")
# 买入操作
for i, symbol in enumerate(tickers):
    qty_diff = difference[i]
    if qty_diff > 0:
        # 买入股票
        # present quantity < optimal quantity : buy

        create_order(symbol, qty_diff, "buy", "market", "day")

#### to see the result of orders:
>   [alpaca orders](https://app.alpaca.markets/orders)
>   [alpaca positions](https://app.alpaca.markets/positions)


#### to see if this is a time for open trading

In [14]:
BASE_URL = "https://paper-api.alpaca.markets"
time_url = BASE_URL+"/v2/calendar"

api = tradeapi.REST(key_id=API_KEY,secret_key=SECRET_KEY,base_url=BASE_URL)

import datetime

# Check if the market is open now.
clock = api.get_clock()
print('The market is {}'.format('open.' if clock.is_open else 'closed.'))

# Check when the market was open on Dec. 1, 2018
date = "2023-09-15"
calendar = api.get_calendar(start=date, end=date)[0]
print('The market opened at {} and closed at {} on {}.'.format(
    calendar.open,
    calendar.close,
    date
))

The market is closed.
The market opened at 09:30:00 and closed at 16:00:00 on 2023-09-15.
